# Práctica: MANOVA

Eres parte de un equipo de investigación que evalúa la rehabilitación neuropsicológica de pacientes que entraron en estado de coma. Actualmente, se te ha pedido que indagues sobre características sociodemográficas que puedan potencialmente influir en la rehabilitación de estos pacientes. Para ello se te brindan los siguientes datos:

In [58]:
import pandas as pd
from sklearn.preprocessing import PowerTransformer

# Load and prepare data
df = pd.read_csv("https://socialsciences.mcmaster.ca/jfox/Books/Applied-Regression-3E/datasets/Wong.txt", sep=r'\s{1,}', engine='python')
df.dropna(inplace=True)
category = pd.cut(df.age,bins=[0,17,65,99],labels=['minor','adult','elder'])
df.drop(columns=['id','duration', 'days', 'age'], inplace=True)
wong_data = pd.get_dummies(df, drop_first = True)
wong_data.insert(3,'age_group',category)

# Enforce normality for teaching purposes
scaler = PowerTransformer()
scaled_data = scaler.fit_transform(wong_data[['piq','viq']])
wong_data['piq'] = scaled_data[:,0]
wong_data['viq'] = scaled_data[:,1]
wong_data

,piq,viq,sex_Male,age_group
0,0.018012,-0.377590,1,adult
1,0.534757,-1.328667,1,adult
2,0.534757,1.440707,1,adult
3,-2.075023,-1.671548,1,adult
4,-1.422233,-1.671548,1,adult
...,...,...,...,...
326,0.904403,-0.760845,1,adult
327,0.279997,-0.452864,1,adult
328,0.084214,0.612245,1,minor
329,0.904403,0.060611,1,minor


1. Evalúa el supuesto de normalidad univariante y multivariante de las variables dependientes. 

In [59]:
import pingouin as pg
pg.multivariate_normality(wong_data[['piq','viq']],)

HZResults(hz=1.0706313106107561, pval=0.05330265179136906, normal=True)

In [60]:
pg.normality(wong_data['piq'])

,W,pval,normal
piq,0.996002,0.568481,True


2. Evalúa el supuesto de homocedasticidad univariante y multivariante de las variables dependientes.

In [63]:
#pg.box_m(data = wong_data, dvs=['piq','viq'], group='sex_Male')
pg.box_m(data = wong_data, dvs=['piq','viq'], group='age_group')

,Chi2,df,pval,equal_cov
box,5.67262,6.0,0.460844,True


3. Verifica que las variables dependientes se encuentren correlacionadas, pero no presenten multicolinealidad extrema. 

In [64]:
wong_data[['piq','viq']].corr()

,piq,viq
piq,1.0000,0.6429
viq,0.6429,1.0000


In [65]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

statistic, p_value = calculate_bartlett_sphericity(wong_data[['piq','viq']])
print("Chi-squared: ", statistic)
print("p-value: ", p_value)

Chi-squared:  177.16921642866464
p-value:  1.01136845706425e-40


4. Implementa un modelo MANOVA que satisfaga la ecuación $ Y_{piq} + Y_{viq} = X_{sex} + X_{age \ group} $. ¿Qué puedes concluir?

In [66]:
from statsmodels.multivariate.manova import MANOVA

maov = MANOVA.from_formula('piq + viq  ~ sex_Male * age_group', data=wong_data)
print(maov.mv_test())

                  Multivariate linear model
                                                             
-------------------------------------------------------------
       Intercept        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.9937 2.0000 324.0000  1.0205 0.3616
         Pillai's trace 0.0063 2.0000 324.0000  1.0205 0.3616
 Hotelling-Lawley trace 0.0063 2.0000 324.0000  1.0205 0.3616
    Roy's greatest root 0.0063 2.0000 324.0000  1.0205 0.3616
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
       age_group        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.9813 4.0000 648.0000  1.5383 0.1894
         Pillai's trace 0.0188 4.0000 650.0000  1.5421 0.1883
 Hotelling-Lawley trace 0.

5. Verifica si las diferencias significativas encontradas con tu modelo MANOVA se replican con cada variable dependiente por separado. ¿Qué puedes concluir?

In [70]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

# generate model for linear regression
my_model_1 = smf.ols(formula='piq ~ sex_Male*age_group', data=wong_data)
my_model_2 = smf.ols(formula='viq ~ sex_Male*age_group', data=wong_data)

# show anova table
print(sm.stats.anova_lm(my_model_1.fit()))
print(sm.stats.anova_lm(my_model_2.fit()))

                       df      sum_sq   mean_sq         F    PR(>F)
age_group             2.0    0.398722  0.199361  0.198371  0.820164
sex_Male              1.0    0.794283  0.794283  0.790340  0.374655
sex_Male:age_group    2.0    3.185660  1.592830  1.584923  0.206544
Residual            325.0  326.621335  1.004989       NaN       NaN
                       df      sum_sq   mean_sq         F    PR(>F)
age_group             2.0    1.214393  0.607197  0.602760  0.547910
sex_Male              1.0    0.291551  0.291551  0.289420  0.590961
sex_Male:age_group    2.0    2.101741  1.050871  1.043192  0.353505
Residual            325.0  327.392315  1.007361       NaN       NaN
